In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

In [2]:
port_build_date = "2024-11-18" # Change to the date we need later
port_build_end = "2024-11-19" # Has to be one day later

exchange_ticker = yf.Ticker('CADUSD=x')
exchange_rate_df = exchange_ticker.history(start=port_build_date, end=port_build_end, interval='1d') 
exchange_rate = exchange_rate_df["Close"].mean()
def build_portfolio(stocks):
    '''
    What to do:
     - Get stock price
     - Get what currency it is dealing with
     - If it is in usd, exchange it over with the exchange rate
     - See how many shares to purchase with the weighted money (MINUS THE FEE) and get the value of those shares
     - At the end: Append to portfolio:
        - Ticker
        - Individual Price 
        - Currency 
        - Num Shares 
        - Total Value 
        - Weight
    - Show that money adds to 1000000 and weight adds to 100.
    '''
    n = 1000000  # budget
    portfolio = [] 
    remaining_budget = 1000000  # budget, goes down as we buy stocks
    for stock, row in stocks.iterrows():
        
        # Grabbing Data
        stock = yf.Ticker(row["stock"])
        weight = row["weight"]
        stock_data = stock.history(start=port_build_date, end=port_build_end, interval='1d') 
        price = stock_data['Close'].mean() # price on the specified day
        
        
        # Exchanging currency if needed
        if stock.info["currency"] == "USD":
            price_cad = price * (1/exchange_rate)
        else: price_cad = price
            
        # Getting number of shares with fee
        shares_cost = n*weight
        shares = shares_cost / price_cad
        fee = max(3.95, shares * 0.001)
        fee_cost = shares_cost - fee
        shares = fee_cost / price_cad
        value = shares * price_cad
        
        # Adding our stock to portfolio
        portfolio.append({
            'Ticker': row["stock"],
            'Price': round(price_cad, 2),
            'Currency': stock.fast_info["currency"],
            'Shares': round(shares, 2), 
            'Value': round(value, 2),
            'Weight': row["weight"]  
        })
        
        # Note: Find a way to fix floating point rounding 
    
    # Creating our final dataframe  
    Portfolio_Final = pd.DataFrame(portfolio)
    Portfolio_Final.index += 1
    return Portfolio_Final



In [3]:
stocks_test = pd.DataFrame({
    'stock': [
        'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'META', 'NFLX', 'SHOP', 'JNJ', 
        'V', 'JPM', 'PG', 'XOM', 'UNH', 'MA', 'HD', 'CVX', 'PFE', 'KO', 'PEP', 'ABBV', 'TD.TO'
    ],
    'weight': [
        0.12, 0.11, 0.10, 0.08, 0.07, 0.06, 0.05, 0.04, 0.04, 0.04,
        0.03, 0.03, 0.03, 0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02,
        0.02, 0.01, 0.01
    ]
})
port_test = build_portfolio(stocks_test)
print(port_test)
print(port_test["Value"].sum())
print(port_test["Weight"].sum())

   Ticker    Price Currency  Shares      Value  Weight
1    AAPL   319.66      USD  375.39  119996.05    0.12
2    MSFT   582.85      USD  188.72  109996.05    0.11
3   GOOGL   245.75      USD  406.90   99996.05    0.10
4    AMZN   282.76      USD  282.91   79996.05    0.08
5    TSLA   474.88      USD  147.40   69996.05    0.07
6    NVDA   196.47      USD  305.36   59996.05    0.06
7    META   777.21      USD   64.33   49996.05    0.05
8    NFLX  1187.47      USD   33.68   39996.05    0.04
9    SHOP   148.43      USD  269.46   39996.05    0.04
10    JNJ   216.97      USD  184.34   39996.05    0.04
11      V   437.61      USD   68.54   29996.05    0.03
12    JPM   343.51      USD   87.32   29996.05    0.03
13     PG   239.37      USD  125.31   29996.05    0.03
14    XOM   168.66      USD  177.85   29996.05    0.03
15    UNH   826.62      USD   36.29   29996.05    0.03
16     MA   731.27      USD   27.34   19996.05    0.02
17     HD   575.39      USD   34.75   19996.05    0.02
18    CVX 